In [ ]:
import utils

import os
import glob

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from natsort import natsorted
import re
import shutil
import skimage

from tqdm.notebook import tqdm_notebook

os.environ["OPENCV_IO_MAX_result_PIXELS"] = pow(2,40).__str__()

import cv2

def split_label_dims(img_path):
    img = cv2.imread(img_path,0)
    final_image = np.zeros_like(img)

    for value in [1, 2, 3, 4]:
        single_channel_img = np.where(img == value, 1, 0)
        final_image = np.dstack((final_image, single_channel_img))


    final_image = final_image[:, :, 1:]

    return final_image

def split_label_dims_highres(img):
    final_image = np.zeros_like(img)

    for value in [1, 2, 3, 4]:
        single_channel_img = np.where(img == value, 1, 0)
        final_image = np.dstack((final_image, single_channel_img))


    final_image = final_image[:, :, 1:]

    return final_image


Steps for multiplex images:

- Downscale to your desired resolution
- Blend multiplex images
- Group them in their corresponding sets
- Align them between each other or with H&E as reference
- Align each channel independently based on the previous output
- Build the 3D in Napari

### Downscale

In [ ]:
#utils.resize_qptiff_files("./data/multiplex", "32")

### Blend

In [ ]:
utils.blend_multiplex("./data/multiplex/processed_32", invert_image=False)
utils.group_multiplex_files()

### Croping section (Manual Step to avoid having unnecesarily large images) 

In [ ]:
def crop_and_save(img_path, cropping_values):
    img = cv2.imread(img_path)
    cropped_image = img[cropping_values[0]:cropping_values[1], cropping_values[2]:cropping_values[3]]
    cv2.imwrite(img_path, cropped_image)

In [ ]:
import json

# Load the dictionary from the JSON file (manually annotated cropping regions)
with open('./data/cropping_dict.json', 'r') as json_file:
    cropping_dict_loaded = json.load(json_file)

print("Cropping dictionary loaded from cropping_dict.json")

In [ ]:
files = glob.glob("./data/multiplex/processed_32/*.tif")

for key in cropping_dict_loaded:
    search_key = key.split(".")[0]
    cropping_values = cropping_dict_loaded[key]
    counter = 0
    for file in files: 
        if search_key in file:
             crop_and_save(file, cropping_values)
             counter += 1

    assert counter == 8, "Something went wrong images cropped should be 8"


### Align and check alignment

After aligning shapes should be equal to the HE registered

In [ ]:
# Aligning for now runs in matlab
utils.multiplex_alignment_report("./data/test_alignment_all_2", verbose=False)

### Apply affine transform channel-wise

In [ ]:
utils.apply_affine_channel_wise(dir="./data/multiplex/processed_32/*.tif")

### Visualize only multiplex

In [ ]:
utils.visualize_multiplex()

### Combine Multiplex and HE (OOM error try reshaping)

In [ ]:
def black_pad_image(image, divisor=128):
    """
    Pads an image with the required size to predict.
    """
    # Calculate the amount of padding needed
    height, width = image.shape[:2]
    pad_height = (divisor - height % divisor) % divisor
    pad_width = (divisor - width % divisor) % divisor

    # Create a white canvas with the desired dimensions
    white_canvas = np.zeros((height + pad_height, width + pad_width), dtype=np.uint8)

    # Place the original image on the white canvas
    white_canvas[:height, :width] = image

    return white_canvas


multiplex_files = glob.glob("./data/blended_multiplex/*.tif")

multiplex_numbers = []

for file in multiplex_files: 
    multiplex_numbers.append(os.path.basename(file).split(".")[0]) 

image_slides = multiplex_numbers.copy()

for path in glob.glob("./data/predictions/*.png"):
    match = re.search(r'_(\d+)_Scan', path)
    number = match.group(1)
    image_slides.append(number)

final_stack = {"DAPI":[], "Opal_480":[], "Opal_520":[], "Opal_570":[], "Opal_620":[], "Opal_690":[], "Opal_780":[], "Sample_AF":[], "Stroma":[], "Adipocytes":[], "Epithelium":[]}
multiplex_channels = ["DAPI", "Opal_480", "Opal_520", "Opal_570", "Opal_620", "Opal_690", "Opal_780", "Sample_AF"]


for slide in tqdm_notebook(natsorted(image_slides)):
    if slide not in multiplex_numbers:
        for he_path in glob.glob("./data/predictions/*.png"):
            if slide in he_path:
                img = split_label_dims(he_path)
                img = np.uint8(img[:,:,0])
                img = cv2.resize(img, (img.shape[1]//2, img.shape[0]//2))
                img = np.int32(img)
                img = skimage.morphology.label(img)
                img = skimage.morphology.remove_small_objects(img, 32)
                img = np.where(img > 0, 1, 0)
                
                final_stack["Epithelium"].append(img)
                
                img = split_label_dims(he_path)
                img = np.uint8(img[:,:,2])
                img = cv2.resize(img, (img.shape[1]//2, img.shape[0]//2))
                img = np.where(img > 0, 1, 0)
                final_stack["Stroma"].append(img)

                img = split_label_dims(he_path)
                img = np.uint8(img[:,:,3])
                img = cv2.resize(img, (img.shape[1]//2, img.shape[0]//2))
                img = np.where(img > 0, 1, 0)
                final_stack["Adipocytes"].append(img)
                
                # Include a zerolike matrix in all multiplex image 
                final_stack["DAPI"].append(np.zeros_like(img))
                final_stack["Opal_480"].append(np.zeros_like(img))
                final_stack["Opal_520"].append(np.zeros_like(img))
                final_stack["Opal_570"].append(np.zeros_like(img))
                final_stack["Opal_620"].append(np.zeros_like(img))
                final_stack["Opal_690"].append(np.zeros_like(img))
                final_stack["Opal_780"].append(np.zeros_like(img))
                final_stack["Sample_AF"].append(np.zeros_like(img))
                break
    else:
        for channel in multiplex_channels:
            for file in glob.glob("./data/registered_multiplex/*.tif"):
                if channel in file and slide in file:
                    image = cv2.imread(file, 0)
                    padded_image = black_pad_image(image, divisor=128)
                    padded_image = cv2.resize(padded_image, (padded_image.shape[1]//2, padded_image.shape[0]//2))
                    final_stack[channel].append(padded_image)

        final_stack["Stroma"].append(np.zeros_like(padded_image))
        final_stack["Adipocytes"].append(np.zeros_like(padded_image))
        final_stack["Epithelium"].append(np.zeros_like(padded_image))

                    

In [ ]:
import napari

colormaps = ["blue", "bop orange", "bop purple", "cyan", "gray", "green", "yellow", "magenta", "turbo", "yellow", "orange"]

viewer = napari.Viewer()
for channel, color in zip(final_stack, colormaps):
    # print(channel)
    stack = final_stack[channel]
    stack = np.array(stack)
    viewer.add_image(stack, name=channel, scale=(20,8,8), colormap=color, blending="additive")

In [ ]:
import glob
import cv2
import matplotlib.pyplot as plt


for path in glob.glob("./test_predictions/*.png"):
    colors = ["black", "green", "red", "blue", "yellow"]
    imin=0
    imax=len(colors)
    cmap = plt.cm.colors.ListedColormap(colors)
    img = cv2.imread(path, 0)
    plt.figure(figsize=(15,15))
    plt.title(path)
    plt.imshow(img, cmap=cmap, vmin=imin, vmax=imax,interpolation="nearest")
    plt.axis("off")
    plt.show()

# HE high res

In [ ]:
import cv2
import matplotlib.pyplot as plt
import glob
import napari 


he_stack = {"Stroma":[], "Adipocytes":[], "Epithelium":[], "Blood Vessels":[]}
he_paths = glob.glob("./data/bck_predictions/*.png")

for he_path in tqdm_notebook(he_paths):

    img = cv2.imread(he_path,0)
    img = cv2.resize(img, (img.shape[1]//16, img.shape[0]//16))

    splitted_img = split_label_dims_highres(img)

    epithelium = np.uint8(splitted_img[:,:,0])
    epithelium = np.int32(epithelium)
    epithelium = skimage.morphology.label(epithelium)
    epithelium = skimage.morphology.remove_small_objects(epithelium, 64)
    epithelium = np.where(epithelium > 0, 1, 0)

    he_stack["Epithelium"].append(epithelium)

    bv = np.uint8(splitted_img[:,:,1])
    bv = np.int32(bv)
    bv = skimage.morphology.label(bv)
    bv = skimage.morphology.remove_small_objects(bv, 64)
    bv = np.where(bv > 0, 1, 0)
    he_stack["Blood Vessels"].append(bv)

    stroma = np.uint8(splitted_img[:,:,2])
    stroma = skimage.morphology.label(stroma)
    stroma = skimage.morphology.remove_small_objects(stroma, 64)
    stroma = np.where(stroma > 0, 1, 0)
    he_stack["Stroma"].append(stroma)

    fat = np.uint8(splitted_img[:,:,3])
    fat = skimage.morphology.label(fat)
    fat = skimage.morphology.remove_small_objects(fat, 64)
    fat = np.where(fat > 0, 1, 0)
    he_stack["Adipocytes"].append(fat)

In [ ]:
import napari

colormaps = ["blue", "yellow", "green", "red"]

viewer = napari.Viewer()
for channel, color in zip(he_stack, colormaps):
    # print(channel)
    stack = he_stack[channel]
    stack = np.array(stack)
    viewer.add_image(stack, name=channel, scale=(20,8,8), colormap=color, blending="additive")

In [ ]:
def remove_small_objects(img):
        binary = np.copy(img)
        binary[binary>0] = 1
        labels = skimage.morphology.label(binary)
        labels_num = [len(labels[labels==each]) for each in np.unique(labels)]
        rank = np.argsort(np.argsort(labels_num))
        index = list(rank).index(len(rank)-2)
        new_img = np.copy(img)
        new_img[labels!=index] = 0
        return new_img

In [ ]:
he_paths = glob.glob("./data/predictions/*.png")
test = []
for he_path in tqdm_notebook(he_paths):
    img = cv2.imread(he_path,0)
    img = cv2.resize(img, (img.shape[1]//8, img.shape[0]//8))

    splitted_img = split_label_dims_highres(img)

    epithelium = np.uint8(splitted_img[:,:,0])
    epithelium = np.int32(epithelium)
    epithelium = skimage.morphology.label(epithelium)
    epithelium = skimage.morphology.remove_small_objects(epithelium, 256)
    epithelium = np.where(epithelium > 0, 2, 0)
    test.append(epithelium.astype("int64"))

test = np.array(test)

In [ ]:
import napari
viewer = napari.Viewer()
viewer.add_labels(test, scale=(8,8,8), blending='additive')